# IMDB 영화 리뷰 감성 분석
- 테스트 데이터 25,000건 중 10,000건은 ~로 활용
- Conv1D + LSTM

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
seed = 2021
np.random.seed(seed)
tf.random.set_seed(seed)

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [5]:
from tensorflow.keras.datasets import imdb
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=None)
X_train.shape, X_test.shape

17465344/17464789 [==============================] - 2s 0us/step


((25000,), (25000,))

In [6]:
index_dict = {}
for key, value in imdb.get_word_index().items():
    index_dict[value] = key
len(index_dict)

1646592/1641221 [==============================] - 0s 0us/step


88584

In [7]:
' '.join(index_dict[s] for s in X_train[2])

"the as there in at by br of sure many br of proving no only women was than doesn't as you never of hat night that with ignored they bad out superman plays of how star so stories film comes defense date of wide they don't do that had with of hollywood br of my seeing fan this of pop out body shots in having because cause it's stick passing first were enjoys for from look seven sense from me superimposition die in character as cuban issues but is you that isn't one song just is him less are strongly not are you that different just even by this of you there is eight when it part are film's love film's 80's was big also light don't wrangling as it in character looked cinematography so stories is far br man acting"

In [9]:
# 뉴스 기사의 길이
print('영화평 최대 길이:', max(len(l) for l in X_train))
print('영화평 평균 길이:', sum(map(len, X_train))/len(X_train))

영화평 최대 길이: 2494
영화평 평균 길이: 238.71364


## Conv1D와 LSTM으로 IMDB 리뷰 감성 분류
- 단어 빈도 수: 5000 (총 단어수 : 88,584)
- 문장의 단어 수: 500단어 (최대: 2,494)
- Test data 중 10000개는 검증 데이터로

In [11]:
from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.layers import Conv1D, Dropout, MaxPooling1D
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [12]:
# 단어 빈도 수: 5000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=5000)

In [13]:
# 문장의 단어 수: 500단어
max_len = 500
X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)

In [14]:
#  Test data 중 10000개는 검증 데이터로
from sklearn.model_selection import train_test_split
X_test, X_val, y_test, y_val = train_test_split(
    X_test, y_test, stratify=y_test, test_size=0.4, random_state=seed
)
X_test.shape, X_val.shape, y_test.shape, y_val.shape

((15000, 500), (10000, 500), (15000,), (10000,))

## 모델 정의/설정/학습/평가

In [15]:
model = Sequential([
    Embedding(5000, 120),
    Dropout(0.5),
    Conv1D(64, 5, activation='relu'),
    MaxPooling1D(pool_size=4),
    LSTM(60),
    Dense(1, activation='sigmoid')
])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 120)         600000    
_________________________________________________________________
dropout (Dropout)            (None, None, 120)         0         
_________________________________________________________________
conv1d (Conv1D)              (None, None, 64)          38464     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, None, 64)          0         
_________________________________________________________________
lstm (LSTM)                  (None, 60)                30000     
_________________________________________________________________
dense (Dense)                (None, 1)                 61        
Total params: 668,525
Trainable params: 668,525
Non-trainable params: 0
__________________________________________________

In [16]:
model.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy']
)

In [17]:
# Callback 함수
model_file = 'model/best_imdb_conv1d_lstm.h5'
mc = ModelCheckpoint(model_file, save_best_only=True, verbose=1)
es = EarlyStopping(patience=5)

In [18]:
history = model.fit(
    X_train, y_train, batch_size=100, epochs=50,
    validation_data=(X_val, y_val), verbose=0, callbacks=[mc, es]
)


Epoch 00001: val_loss improved from inf to 0.31088, saving model to model\best_imdb_lstm.h5

Epoch 00002: val_loss improved from 0.31088 to 0.27086, saving model to model\best_imdb_lstm.h5

Epoch 00003: val_loss did not improve from 0.27086

Epoch 00004: val_loss did not improve from 0.27086

Epoch 00005: val_loss did not improve from 0.27086

Epoch 00006: val_loss did not improve from 0.27086


KeyboardInterrupt: 

In [19]:
best_model = load_model(model_file)
best_model.evaluate(X_test, y_test)

469/469 [==============================] - 9s 20ms/step - loss: 0.2701 - accuracy: 0.8861


[0.2700861096382141, 0.8860666751861572]